# Export


In [ ]:
from keras.models import load_model
import utils_io
import numpy as np
import pandas as pd

# Prepare data


TEST DATA


In [ ]:
path_test_data = "data_set/test.csv"

test_data = utils_io.load_csv_data(path_test_data)
# Debugging: Check the type of test_data

# Print the loaded data
if isinstance(test_data, pd.DataFrame):
    print(f"dimension train data: {test_data.shape}")
else:
    print("test_data is not a DataFrame")

target data


In [ ]:
path_target_data = "data_set/sample_submission.csv"

target_data = utils_io.load_csv_data(path_target_data)

# Print the loaded data
if isinstance(target_data, pd.DataFrame):
    print(f"dimension train data: {target_data.shape}")
else:
    print("test_data is not a DataFrame")

In [ ]:
# Drop the Id and sales price column
test_data = test_data.drop(columns=["Id"])
print(test_data.shape)

Clean numbers NA to na.nan


In [ ]:
import pandas as pd
import numpy as np


def auto_clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df_cleaned = df.copy()

    for column in df_cleaned.columns:
        values = df_cleaned[column].dropna().unique()

        # Pokud vše je číslo nebo 'NA' → převedeme na float
        can_be_numeric = True
        for v in values:
            if isinstance(v, str) and v.strip().upper() == "NA":
                continue
            try:
                float(v)
            except:
                can_be_numeric = False
                break

        if can_be_numeric:
            # Převést "NA" na np.nan a sloupec na float
            df_cleaned[column] = df_cleaned[column].replace("NA", np.nan)
            df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors="coerce")
        else:
            # ponecháme jako kategorii se stringy
            df_cleaned[column] = df_cleaned[column].astype(str).str.strip().str.upper()

    return df_cleaned


test_data = auto_clean_columns(test_data)
print(test_data)

## Normalization


In [ ]:
import json
import re

json_path = "data_set/generated_description.json"

generated_description = json.load(open(json_path))


for column in test_data.columns:
    for index, value in test_data[column].items():
        # if value == "NA":
        #     print(column, index, value)
        if isinstance(value, str):
            try:
                test_data.at[index, column] = generated_description[column]["items"][
                    value.strip().upper()
                ]["id"]
            except:
                print(column, index, value, isinstance(value, str))

In [ ]:
print(test_data.iloc[:10])

In [ ]:
mean = test_data.mean(axis=0)
# print(mean)
std = test_data.std(axis=0)
# print(std)

train_data = test_data - mean
train_data = test_data / std

In [ ]:
train_data = train_data.astype("float32")
target_data = target_data.astype("float32")

# Import model


In [ ]:
model = load_model("house_prices_0.keras")

In [ ]:
print(train_data.iloc[0])

In [ ]:
for i in range(10):
    predictions = model.predict(train_data.iloc[[i]])
    target_value = target_data.iloc[i, 1]

    diff = int(target_value - predictions[0][0])
    pred = int(predictions[0][0])
    target = int(target_value)

    print(
        f"diff = {format(diff, ',').replace(',', ' ')} USD | "
        f"prediction = {format(pred, ',').replace(',', ' ')} USD | "
        f"target = {format(target, ',').replace(',', ' ')} USD"
    )

In [ ]:
target_data.iloc[2, 1]